In [200]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import date
from dateutil.relativedelta import relativedelta
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In this notebook I'm all insights about data quality I gained in "Data quality checks and data claning" and "Exploratory analysis".

In [2]:
#importing data in original shape
customers = pd.read_csv("data_updated/customers.csv")
districts = pd.read_csv("data_updated/districts.csv")
transactions = pd.read_csv("data_updated/transactions.csv")

#splitting customers and transactions into train and test
customers_train = customers[customers.SET_SPLIT == 'TRAIN']
customers_test = customers[customers.SET_SPLIT == 'TEST']

transactions = transactions.merge(customers[['ACCOUNT_ID','SET_SPLIT']], on='ACCOUNT_ID')
transactions_train = transactions[transactions.SET_SPLIT == 'TRAIN']
transactions_test = transactions[transactions.SET_SPLIT == 'TEST']

# drop SET_SPLIT column
customers_train.drop(columns=['SET_SPLIT'], inplace=True)
customers_test.drop(columns=['SET_SPLIT'], inplace=True)
transactions_train.drop(columns=['SET_SPLIT'], inplace=True)
transactions_test.drop(columns=['SET_SPLIT'], inplace=True)

C:\Users\katar\Anaconda3\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


From the previous notebooks I have learned, that I need to do some data cleaning/preprocessing tasks. I need to combine customers, districts and transactions tables, and before combining, I need to select relevant features from districts table and do feature engineering for transactions. From my exploration in previous notebooks, I found that following features can have predictive power:
* from customers table: BIRTH_DT (transformed to AGE),
* from districts table: N_INHAB, N_CITIES, URBAN_RATIO, AVG_SALARY
* from transactions table: avergae of monthly credit transsactions, average of monthly debit transactions

I realize it probably can be put into sklearn pipeline, but I couldn't make it work in time.

In [201]:
# transforming train data

df1 = customers_train
df2 = districts
df3 = transactions_train

#dropping nan values
df1 = df1[~df1.LOAN.isna()]

# set aside target variable
y = df1.LOAN
# keep only relevant columns
df1 = df1[['ACCOUNT_ID', 'DISTRICT_ID',  'BIRTH_DT']]
df2 = df2[['DISTRICT_ID', 'N_INHAB', 'N_CITIES', 'URBAN_RATIO', 'AVG_SALARY']]
df3 = df3[['ACCOUNT_ID','DATE','AMOUNT','TYPE']]

# transform BIRTH_DT into AGE
df1['AGE'] = [relativedelta(date.today(), dt.strptime(str(x), '%Y%m%d')).years for x in df1.BIRTH_DT]
df1.drop(columns=['BIRTH_DT'], inplace=True)

X = df1.merge(df2, on='DISTRICT_ID')

# transform transactions data
#dropping duplicates, because duplicates were found in exploratory analysis
df3.drop_duplicates(inplace=True)

# define function to clean DATE
def add_zero(a):
    if len(a) == 7:
        a = '0' + a
    return a


df3['DATE2'] = [dt.strptime(add_zero(str(x)), '%d%m%Y') for x in df3.DATE]
df3['YEAR'] = [x.year for x in df3.DATE2]
df3['MONTH'] = [x.month for x in df3.DATE2]


df3_agg = df3.groupby(['ACCOUNT_ID','YEAR','MONTH','TYPE']).agg({'AMOUNT':{'sum'}})
df3_agg.columns = df3_agg.columns.get_level_values(1)
df3_agg.reset_index()

df3_agg2 = df3_agg.groupby(['ACCOUNT_ID','TYPE']).agg({'sum':{'mean'}})
df3_agg2.columns = df3_agg2.columns.get_level_values(1)
df3_agg2.reset_index()

df3_p = df3_agg2.pivot_table(index=['ACCOUNT_ID'], columns= ['TYPE'], values=['mean'])
df3_p.fillna(0)
df3_p.columns = df3_p.columns.get_level_values(1)
df3_p.reset_index()

X = X.merge(df3_p, on='ACCOUNT_ID')

# drop merge keys

X.drop(columns=['ACCOUNT_ID','DISTRICT_ID'], inplace=True)
X.head()


C:\Users\katar\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\katar\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\katar\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

,AGE,N_INHAB,N_CITIES,URBAN_RATIO,AVG_SALARY,CREDIT,WITHDRAWAL
0,52,70699,4,65.3,8968,4224.358696,4205.906977
1,87,70699,4,65.3,8968,4942.553333,4843.533333
2,42,70699,4,65.3,8968,20344.561538,20470.150000
3,43,70699,4,65.3,8968,13354.123913,12445.090909
4,76,70699,4,65.3,8968,10459.337931,11514.016667


In [202]:
y_train = y.to_numpy()
X_train = X.to_numpy()

I will try to train RandomForest.

In [214]:
skl_rf=RandomForestClassifier(class_weight = {1:5.6, 0:1}, verbose=0)
skl_rf.fit(X_train, y_train)
y_train_pred = skl_rf.predict(X_train)
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_train_pred, pos_label=1)
print("AUC: {}".format(metrics.auc(fpr, tpr)))
metrics.confusion_matrix(y_train, y_train_pred,labels=[1,0])

AUC: 0.9506260551135732


C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curr

array([[ 478,   51],
       [   7, 2985]], dtype=int64)

This AUC is much higher than expected, it's most likely due to overfitting. I will proceed to tune hyperparameters with use of cross validation.


In [215]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()
param_grid = {'bootstrap': [True, False],
 'max_depth': [2,3,4,5],
 'n_jobs': [-1],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [15, 20, 30],
 'n_estimators': [20],
 'class_weight': [{1:5.6, 0:1}]}

GS = GridSearchCV(rf, param_grid, refit=True, cv=4, return_train_score=True)

In [216]:
GS.fit(X_train, y_train)

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link fo

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link fo

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link fo

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link fo

...

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your 

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your 

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link fo

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:793: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release no

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [217]:
GS.best_estimator_


RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 5.6},
                       criterion='gini', max_depth=5, max_features='sqrt',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=20,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=20, n_jobs=-1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [218]:
GS.best_score_

0.4274353876739563

In [219]:
best_rf = GS.best_estimator_
y_train_pred = best_rf.predict(X_train)
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_train_pred, pos_label=1)
metrics.auc(fpr, tpr)

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


0.7910331773197335

In [224]:
metrics.confusion_matrix(y_train,y_train_pred)

array([[2556,  436],
       [ 144,  385]], dtype=int64)

GridSearchCV didn't manage to find a better model. GS.best_score_ is mean result of model performance over k-folds, which it worse than random classifier. At this stage I would go back to features. Possible ways for improvement: some of the features from districts table are correlated, maybe it would be better to remove some of them. Transactions table is not fully used, there are much more possible features that could be derived, e.g. amount / count / std of transactions, grouped by not only TYPE, but also OPERATION. There are even examples of using time-frequency features, which we could potentially derive thank to DATE column. For sake of completeness I'll check performance on test data.

In [225]:
# repeat preprocessing on test

df1 = customers_test
df2 = districts
df3 = transactions_test

#dropping nan values
df1 = df1[~df1.LOAN.isna()]

# set aside target variable
y = df1.LOAN
# keep only relevant columns
df1 = df1[['ACCOUNT_ID', 'DISTRICT_ID',  'BIRTH_DT']]
df2 = df2[['DISTRICT_ID', 'N_INHAB', 'N_CITIES', 'URBAN_RATIO', 'AVG_SALARY']]
df3 = df3[['ACCOUNT_ID','DATE','AMOUNT','TYPE']]

# transform BIRTH_DT into AGE
df1['AGE'] = [relativedelta(date.today(), dt.strptime(str(x), '%Y%m%d')).years for x in df1.BIRTH_DT]
df1.drop(columns=['BIRTH_DT'], inplace=True)

X = df1.merge(df2, on='DISTRICT_ID')

# transform transactions data
#dropping duplicates, because duplicates were found in exploratory analysis
df3.drop_duplicates(inplace=True)

# define function to clean DATE
def add_zero(a):
    if len(a) == 7:
        a = '0' + a
    return a


df3['DATE2'] = [dt.strptime(add_zero(str(x)), '%d%m%Y') for x in df3.DATE]
df3['YEAR'] = [x.year for x in df3.DATE2]
df3['MONTH'] = [x.month for x in df3.DATE2]


df3_agg = df3.groupby(['ACCOUNT_ID','YEAR','MONTH','TYPE']).agg({'AMOUNT':{'sum'}})
df3_agg.columns = df3_agg.columns.get_level_values(1)
df3_agg.reset_index()

df3_agg2 = df3_agg.groupby(['ACCOUNT_ID','TYPE']).agg({'sum':{'mean'}})
df3_agg2.columns = df3_agg2.columns.get_level_values(1)
df3_agg2.reset_index()

df3_p = df3_agg2.pivot_table(index=['ACCOUNT_ID'], columns= ['TYPE'], values=['mean'])
df3_p.fillna(0)
df3_p.columns = df3_p.columns.get_level_values(1)
df3_p.reset_index()

X = X.merge(df3_p, on='ACCOUNT_ID')

# drop merge keys

X.drop(columns=['ACCOUNT_ID','DISTRICT_ID'], inplace=True)
X.head()


C:\Users\katar\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\katar\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\katar\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

,AGE,N_INHAB,N_CITIES,URBAN_RATIO,AVG_SALARY,CREDIT,WITHDRAWAL
0,66,95616,6,51.4,9307,9608.116667,7462.675000
1,65,95616,6,51.4,9307,22075.791429,21324.747059
2,56,95616,6,51.4,9307,30599.462963,30288.824000
3,86,95616,6,51.4,9307,4501.658621,4772.033333
4,69,95616,6,51.4,9307,6086.159091,6003.111111


In [226]:
X_test = X.to_numpy()
y_test = y.to_numpy()

In [227]:
y_pred = best_rf.predict(X_test)

C:\Users\katar\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [229]:
metrics.confusion_matrix(y_test,y_pred)

array([[567, 181],
       [ 99,  32]], dtype=int64)

In [228]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.5011480997673184